In [ ]:
import ibis
from ibis import _
import os
from osgeo import gdal

from cng.utils import *
from cng.h3 import *

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
install_h3(con)


set_secrets(con)

In [3]:
import os
os.environ['AWS_VIRTUAL_HOSTING']='FALSE'


In [6]:
# could use bash
# ! gdalwarp -t_srs EPSG:4326 -of XYZ /vsicurl/https://data.source.coop/cboettig/mobi/species-richness-all/mobi-species-richness.tif mobi.txt

In [4]:
# os.unlink("mobi.xyz")
from cng.zonal import raster_reprojection 

raster_reprojection("/vsicurl/https://data.source.coop/cboettig/mobi/species-richness-all/mobi-species-richness.tif", "/vsis3/public-data/cache/mobi.xyz", 'EPSG:4326')

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [5]:
# not sure why ibis buildin is failing here??


mobi = (con
  .read_csv("mobi.xyz", 
            delim = ' ', 
            columns = {'X': 'FLOAT', 'Y': 'FLOAT', 'Z': 'INTEGER'})
  .mutate(h8 = h3_latlng_to_cell_string(_.Y, _.X, 8))
  .to_parquet("s3://public-data/cache/mobi.xyz")
)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:


tracts_z8 = (
    con
    .read_parquet("s3://public-social-vulnerability/2022-tracts-h3-z8.parquet")
    .mutate(h8 = _.h8.lower())
)

tracts_mobi = (
    con.read_parquet("s3://public-data/cache/mobi.xyz")
    .inner_join(tracts_z8, "h8")
    .group_by(_.COUNTY)
    .agg(richness = _.Z.mean())
)  


In [7]:
import ibis
from ibis import _
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])


pad_z8 =  con.read_parquet("https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-h3-z8.parquet")
tracts_z8 = con.read_parquet("https://minio.carlboettiger.info/public-social-vulnerability/2022-tracts-h3-z8.parquet").mutate(h8 = _.h8.lower())
mobi = con.read_parquet("https://minio.carlboettiger.info/public-mobi/hex/all-richness-h8.parquet").rename(richness = "Z").select('richness', 'h8')
svi = con.read_parquet("https://minio.carlboettiger.info/public-social-vulnerability/2022/SVI2022_US_tract.parquet").select('FIPS', 'RPL_THEMES')

combined = tracts_z8.inner_join(mobi, "h8").inner_join(pad_z8, "h8").inner_join(svi, "FIPS")




In [8]:
import leafmap.maplibregl as leafmap
m = leafmap.Map()

pmtiles = "https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-us-4.pmtiles"
pmtiles = "https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-us-4.pmtiles"

m.add_pmtiles(pmtiles)
m

Map(calls=[['addControl', ('NavigationControl', {'showCompass': True, 'showZoom': True, 'visualizePitch': Fals…

In [ ]:


hectres_h8 = 737327.598	/ 10000

county_group = (
    combined
    .group_by("COUNTY")
    .agg(richness = _.richness.mean(), 
         area = _.count() * hectres_h8,
         svi = _.RPL_THEMES.mean() 
         )
)

